In [5]:
from transformers import ZeroShotClassificationPipeline, AutoTokenizer, AutoConfig, T5ForSequenceClassification, 
import torch

In [10]:
original_id = 'google-t5/t5-large'
config_path = r'C:\Users\lachl\PycharmProjects\bart-large-mnli-recreation\runs\google-t5\t5-large\2024-06-16--14-29-48\config_checkpoint'
checkpoint_path = r'C:\Users\lachl\PycharmProjects\bart-large-mnli-recreation\runs\google-t5\t5-large\2024-06-16--10-03-52\checkpoints\checkpoint_1\model.safetensors'

In [20]:
config = AutoConfig.from_pretrained(config_path)

id2label = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}

config.label2id = {v:k for k,v in id2label.items()}
config.max_length = 2048

In [23]:

tokenizer = AutoTokenizer.from_pretrained(original_id)
model = T5ForSequenceClassification.from_pretrained(
    checkpoint_path,
    device_map='cuda:0',
    config=config,
)

classifier = ZeroShotClassificationPipeline(model=model, tokenizer=tokenizer)

In [28]:
sequence_to_classify = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing', 'negative sentiment', 'positive sentiment', 'neutral sentiment']
classifier(sequence_to_classify, candidate_labels, multi_label=True)
#{'labels': ['travel', 'dancing', 'cooking'],
# 'scores': [0.9938651323318481, 0.0032737774308770895, 0.002861034357920289],
# 'sequence': 'one day I will see the world'}


{'sequence': 'one day I will see the world',
 'labels': ['travel',
  'positive sentiment',
  'neutral sentiment',
  'negative sentiment',
  'dancing',
  'cooking'],
 'scores': [0.985240638256073,
  0.8247328996658325,
  0.14022280275821686,
  0.09571882337331772,
  0.07555826008319855,
  0.053149327635765076]}